<a href="https://colab.research.google.com/github/brempong21/Data-Science-Salaries/blob/main/tfx%20pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tfx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.6/206.6 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import os
import tensorflow as tf
import tfx
import tensorflow_transform.beam as tft_beam
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict
from tensorflow_transform.tf_metadata import dataset_metadata, schema_utils
import pprint

pp = pprint.pprint
import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

In [3]:
!wget https://archive.ics.uci.edu/ml/datasets/Metro+Interstate+Traffic+Volume.csv

--2024-03-01 13:53:30--  https://archive.ics.uci.edu/ml/datasets/Metro+Interstate+Traffic+Volume.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-03-01 13:53:30 ERROR 404: Not Found.



In [2]:
_pipeline_root = './pipeline/'
_data_root = './data/metro/'
_data_filepath = os.path.join(_data_root, 'data_science_salaries.csv')

In [3]:
!head{_data_filepath}

/bin/bash: line 1: head./data/metro/data_science_salaries.csv: No such file or directory


In [4]:
import pandas as pd

df = pd.read_csv('/content/data/metro/data_science_salaries.csv')
df.head()

,job_title,experience_level,employment_type,work_models,work_year,employee_residence,salary,salary_currency,salary_in_usd,company_location,company_size
0,Data Engineer,Mid-level,Full-time,Remote,2024,United States,148100,USD,148100,United States,Medium
1,Data Engineer,Mid-level,Full-time,Remote,2024,United States,98700,USD,98700,United States,Medium
2,Data Scientist,Senior-level,Full-time,Remote,2024,United States,140032,USD,140032,United States,Medium
3,Data Scientist,Senior-level,Full-time,Remote,2024,United States,100022,USD,100022,United States,Medium
4,BI Developer,Mid-level,Full-time,On-site,2024,United States,120000,USD,120000,United States,Medium


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6599 entries, 0 to 6598
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   job_title           6599 non-null   object
 1   experience_level    6599 non-null   object
 2   employment_type     6599 non-null   object
 3   work_models         6599 non-null   object
 4   work_year           6599 non-null   int64 
 5   employee_residence  6599 non-null   object
 6   salary              6599 non-null   int64 
 7   salary_currency     6599 non-null   object
 8   salary_in_usd       6599 non-null   int64 
 9   company_location    6599 non-null   object
 10  company_size        6599 non-null   object
dtypes: int64(3), object(8)
memory usage: 567.2+ KB


In [6]:
context = InteractiveContext(pipeline_root =_pipeline_root)


In [7]:
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [8]:
artifact = example_gen.outputs['examples'].get()[0]
print(f"split names:{artifact.split_names}")
print(f"artifact uri: {artifact.uri}")

split names:["train", "eval"]
artifact uri: ./pipeline/CsvExampleGen/examples/1


In [9]:
statistics_gen = tfx.components.StatisticsGen(
    examples = example_gen.outputs['examples']
)
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [10]:
context.show(statistics_gen.outputs['statistics'])

In [11]:
schema_gen = tfx.components.SchemaGen(statistics=statistics_gen.outputs['statistics'])
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [12]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'company_location',STRING,required,,'company_location'
'company_size',STRING,required,,'company_size'
'employee_residence',STRING,required,,'employee_residence'
'employment_type',STRING,required,,'employment_type'
'experience_level',STRING,required,,'experience_level'
'job_title',BYTES,required,,-
'salary',INT,required,,-
'salary_currency',STRING,required,,'salary_currency'
'salary_in_usd',INT,required,,-


,Values
Domain,
'company_location',"'Algeria', 'Andorra', 'Argentina', 'Australia', 'Austria', 'Bahamas', 'Belgium', 'Brazil', 'Canada', 'Central African Republic', 'Chile', 'China', 'Colombia', 'Croatia', 'Czech Republic', 'Denmark', 'Ecuador', 'Estonia', 'Finland', 'France', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Honduras', 'Hong Kong', 'Hungary', 'India', 'Indonesia', 'Iran', 'Ireland', 'Israel', 'Italy', 'Japan', 'Kenya', 'Latvia', 'Lebanon', 'Lithuania', 'Luxembourg', 'Mauritius', 'Mexico', 'Moldova', 'Netherlands', 'New Zealand', 'Nigeria', 'Norway', 'Pakistan', 'Philippines', 'Poland', 'Portugal', 'Puerto Rico', 'Romania', 'Russia', 'Saudi Arabia', 'Singapore', 'Slovenia', 'South Africa', 'South Korea', 'Spain', 'Sweden', 'Switzerland', 'Thailand', 'Turkey', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States', 'Vietnam', 'Armenia', 'Bosnia and Herzegovina', 'Egypt', 'Iraq', 'Malaysia', 'Malta', 'Qatar'"
'company_size',"'Large', 'Medium', 'Small'"
'employee_residence',"'Algeria', 'American Samoa', 'Andorra', 'Argentina', 'Australia', 'Austria', 'Belgium', 'Bolivia', 'Brazil', 'Canada', 'Central African Republic', 'Chile', 'Colombia', 'Costa Rica', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Estonia', 'Finland', 'France', 'Germany', 'Ghana', 'Greece', 'Honduras', 'Hong Kong', 'Hungary', 'India', 'Indonesia', 'Iran', 'Ireland', 'Israel', 'Italy', 'Japan', 'Jersey', 'Kenya', 'Latvia', 'Lebanon', 'Lithuania', 'Luxembourg', 'Malaysia', 'Mauritius', 'Mexico', 'Moldova', 'Netherlands', 'New Zealand', 'Nigeria', 'Norway', 'Pakistan', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Puerto Rico', 'Romania', 'Russia', 'Saudi Arabia', 'Serbia', 'Singapore', 'Slovenia', 'South Africa', 'South Korea', 'Spain', 'Sweden', 'Switzerland', 'Thailand', 'Tunisia', 'Turkey', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States', 'Uzbekistan', 'Vietnam', 'Armenia', 'Bosnia and Herzegovina', 'Bulgaria', 'China', 'Egypt', 'Georgia', 'Iraq', 'Kuwait', 'Malta', 'Qatar'"
'employment_type',"'Contract', 'Freelance', 'Full-time', 'Part-time'"
'experience_level',"'Entry-level', 'Executive-level', 'Mid-level', 'Senior-level'"
'salary_currency',"'AUD', 'BRL', 'CAD', 'CHF', 'CLP', 'DKK', 'EUR', 'GBP', 'HKD', 'HUF', 'ILS', 'INR', 'JPY', 'NOK', 'PHP', 'PLN', 'SGD', 'THB', 'TRY', 'USD', 'MXN', 'ZAR'"
'work_models',"'Hybrid', 'On-site', 'Remote'"


In [13]:
example_validator = tfx.components.ExampleValidator(
    statistics = statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [14]:
context.show(example_validator.outputs['anomalies'])

In [15]:
print("Categorical Columns:")
for column in df.select_dtypes(include=['object']):
    print(column)

print("\nNumerical Columns:")
for column in df.select_dtypes(include=['int', 'float']):
    print(column)

Categorical Columns:
job_title
experience_level
employment_type
work_models
employee_residence
salary_currency
company_location
company_size

Numerical Columns:
work_year
salary
salary_in_usd


In [16]:
module_file = 'data_science_salaries.py'

In [17]:
%%writefile {module_file}

CATEGORICAL_FEATURE_KEYS = ['job_title',
'experience_level',
'employment_type',
'work_models',
'employee_residence',
'salary_currency',
'company_location',
'company_size']

NUMERIC_FEATURE_KEYS = ['work_year','salary','salary_in_usd']

FEATURE_BUCKET_COUNT = ['salary']

LABEL_KEY = 'salary'

def transformed_name(key):
  return key + '_xf'


Writing data_science_salaries.py


In [18]:
module_file_transform = 'data_science_transform.py'

In [21]:
%%writefile {module_file_transform}

import tensorflow as tf
import tensorflow_transform as tft

import data_science_salaries

_NUMERIC_FEATURE_KEYS = data_science_salaries.NUMERIC_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = data_science_salaries.CATEGORICAL_FEATURE_KEYS
_FEATURE_BUCKET_COUNT = data_science_salaries.FEATURE_BUCKET_COUNT
_LABEL_KEY = data_science_salaries.LABEL_KEY
_transformed_name = data_science_salaries.transformed_name

def preprocessing_fn(inputs):
  outputs={}
  for key in _NUMERIC_FEATURE_KEYS:
    outputs[_transformed_name(key)] = tft.scale_to_0_1(inputs[key])

  for key in _CATEGORICAL_FEATURE_KEYS:
    outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(inputs[key])

  outputs[_transformed_name(_LABEL_KEY)] = tft.scale_to_0_1(inputs[_LABEL_KEY])

  return outputs

Overwriting data_science_transform.py


In [22]:
tf.get_logger().setLevel('ERROR')

transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(module_file_transform))

context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 6
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [23]:
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri

os.listdir(transform_graph_uri)

['transformed_metadata', 'metadata', 'transform_fn']

In [26]:
import ml_metadata as mlmd
from ml_metadata.proto import metadata_store_pb2

connection_config = context.metadata_connection_config

store = mlmd.MetadataStore(connection_config)

In [27]:
artifact_types = store.get_artifact_types()

[artifact_type.name for artifact_type in artifact_types]

['Examples',
 'ExampleStatistics',
 'Schema',
 'ExampleAnomalies',
 'TransformGraph',
 'TransformCache']

In [28]:
schema_list = store.get_artifacts_by_type('Schema')

[(f'schema uri: {schema.uri}', f'schema id:{schema.id}') for schema in schema_list]

[('schema uri: ./pipeline/SchemaGen/schema/3', 'schema id:3'),
 ('schema uri: ./pipeline/Transform/pre_transform_schema/5', 'schema id:8'),
 ('schema uri: ./pipeline/Transform/post_transform_schema/5', 'schema id:10'),
 ('schema uri: ./pipeline/Transform/pre_transform_schema/6', 'schema id:16'),
 ('schema uri: ./pipeline/Transform/post_transform_schema/6', 'schema id:18')]

In [29]:
example_anomalies = store.get_artifacts_by_type('ExampleAnomalies')[0]

print(f'Artifact id: {example_anomalies.id}')

Artifact id: 4


In [30]:
anomalies_id_event = store.get_events_by_artifact_ids([example_anomalies.id])[0]

print(anomalies_id_event)

artifact_id: 4
execution_id: 4
path {
  steps {
    key: "anomalies"
  }
  steps {
    index: 0
  }
}
type: OUTPUT
milliseconds_since_epoch: 1709334276646



In [31]:
anomalies_execution_id = anomalies_id_event.execution_id

events_execution = store.get_events_by_execution_ids([anomalies_execution_id])

print(events_execution)

[artifact_id: 2
execution_id: 4
path {
  steps {
    key: "statistics"
  }
  steps {
    index: 0
  }
}
type: INPUT
milliseconds_since_epoch: 1709334276620
, artifact_id: 3
execution_id: 4
path {
  steps {
    key: "schema"
  }
  steps {
    index: 0
  }
}
type: INPUT
milliseconds_since_epoch: 1709334276620
, artifact_id: 4
execution_id: 4
path {
  steps {
    key: "anomalies"
  }
  steps {
    index: 0
  }
}
type: OUTPUT
milliseconds_since_epoch: 1709334276646
]


In [32]:
inputs_to_exval = [event.artifact_id for event in events_execution
                       if event.type == metadata_store_pb2.Event.INPUT]

print(inputs_to_exval)

[2, 3]
